In [7]:
%load_ext autoreload
%autoreload 2

from src.data.mocap import MotionCaptureDataset
from src.models.vae import VariationalAutoencoder
from src.models.svae import SphericalVAE
from src.experiments.mocap import model_args
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from torch.utils.data import ConcatDataset
from src.utils import plot_3d
from pathlib import Path
import optuna
import torch

from plotly.subplots import make_subplots

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
vae_study = optuna.load_study(
    study_name="mocap-07-vae",
    storage="sqlite:///../runs/mocap-07-vae/optuna-storage.db"
)
best_vae_trial = vae_study.best_trial
svae_study = optuna.load_study(
    study_name="mocap-07-svae",
    storage="sqlite:///../runs/mocap-07-svae/optuna-storage.db"
)
best_svae_trial = svae_study.best_trial

In [7]:
dataset = MotionCaptureDataset("07", test=True)
n_features = dataset.n_features
X = dataset.X

In [19]:
vae_params = vae_study.best_trial.params
vae_layer_sizes = [vae_params[f"layer_size_{i+1}"] for i in range(vae_params["n_layers"])]
vae_dropout = vae_study.best_trial.params["dropout"]
vae_number = vae_study.best_trial.number

svae_params = svae_study.best_trial.params
svae_layer_sizes = [svae_params[f"layer_size_{i+1}"] for i in range(svae_params["n_layers"])]
svae_dropout = svae_study.best_trial.params["dropout"]
svae_number = svae_study.best_trial.number

111

In [9]:
svae = SphericalVAE(
    feature_dim=n_features,
    latent_dim=3,
    encoder_params={
        "layer_sizes" : svae_layer_sizes,
        "dropout": svae_dropout,
        "activation_function" : "Tanh"
    },
    decoder_params={
        "layer_sizes" : svae_layer_sizes[::-1],
        "dropout": svae_dropout,
        "activation_function" : "Tanh"
    },
)
svae_state_dict = torch.load(f"../runs/mocap-07-svae/checkpoints/{svae_number:03}.pt")
svae.load_state_dict(svae_state_dict)

<All keys matched successfully>

In [20]:
vae = VariationalAutoencoder(
    feature_dim=n_features,
    latent_dim=3,
    encoder_params={
        "layer_sizes" : vae_layer_sizes,
        "dropout": vae_dropout,
        "activation_function" : "Tanh"
    },
    decoder_params={
        "layer_sizes" : vae_layer_sizes[::-1],
        "dropout": vae_dropout,
        "activation_function" : "Tanh"
    },
)
vae_state_dict = torch.load(f"../runs/mocap-07-vae/checkpoints/{vae_number:03}.pt")
vae.load_state_dict(vae_state_dict)

<All keys matched successfully>

In [11]:
import re
pattern = re.compile("_(\d+):")
classes = [int(pattern.search(s).group(1)) for s in dataset.labels]

In [12]:
fig = make_subplots(
    rows=1,
    cols=3,
    specs=[
        [{"type": "scatter3d"}, {"type": "scatter3d"}, {"type": "scatter3d"}]
    ],
    subplot_titles=(
        "PCA",
        "Ordinary VAE",
        "Spherical VAE",
    ),
    horizontal_spacing=0.01,
    vertical_spacing=0.01,
)

In [13]:
x, y, z = PCA(n_components=3).fit_transform(X).T
plot_3d( x, y, z, classes=classes, fig=fig, row=1, col=1);

In [14]:
output = vae(X)
plot_3d( *output["z"].detach().numpy().T, classes=classes, fig=fig, row=1, col=2);

In [15]:
output = svae(X)
plot_3d( *output["z"].detach().numpy().T, classes=classes, fig=fig, row=1, col=3);

In [16]:
fig

In [93]:
# Run walk dataset
run_dir = "../runs/run-walk"

if __name__ == "__main__":
    
    run_data = MotionCaptureDataset("09", test=True)
    walk_data = MotionCaptureDataset("08", test=True)

    vae = VariationalAutoencoder(
        latent_dim=2, **model_args
    )

    svae = SphericalVAE(
        latent_dim=3, **model_args
    )

    vae_state_dict = torch.load(f"{run_dir}/best_vae.pt", map_location=torch.device('cpu')  )
    vae.load_state_dict( vae_state_dict )

    svae_state_dict = torch.load(f"{run_dir}/best_svae.pt", map_location=torch.device('cpu')  )
    svae.load_state_dict( svae_state_dict )


In [91]:
# KNN classifier using vae
z_run = vae(run_data.X)['z']
z_walk = vae(walk_data.X)['z']
Z = torch.cat([z_run, z_walk]).detach().numpy()

y_run = torch.zeros(z_run.shape[0])
y_walk = torch.ones(z_walk.shape[0])
y = torch.cat([y_run, y_walk])

knn_vae = KNeighborsClassifier(n_neighbors=3)
knn_vae.fit(Z, y)

cross_val_score(knn_vae, Z, y)

array([1., 1., 1., 1., 1.])

In [95]:
# KNN classifier using svae
z_run_svae = svae(run_data.X)['z']
z_walk_svae = svae(walk_data.X)['z']
Z_svae = torch.cat([z_run_svae, z_walk_svae]).detach().numpy()

y_run_svae = torch.zeros(z_run_svae.shape[0])
y_walk_svae = torch.ones(z_walk_svae.shape[0])
y_svae = torch.cat([y_run_svae, y_walk_svae])

knn_svae = KNeighborsClassifier(n_neighbors=3)
knn_svae.fit(Z_svae, y_svae)

cross_val_score(knn_svae, Z_svae, y_svae)




array([1., 1., 1., 1., 1.])

In [12]:
import re
pattern = re.compile("_(\d+):")
run_classes = [int(pattern.search(s).group(1)) for s in run_data.labels]

NameError: name 'run_data' is not defined

In [107]:
# KNN classifier on trials using svae 
knn_run_svae = KNeighborsClassifier(n_neighbors=3)
knn_run_svae.fit(z_run_svae.detach().numpy(), run_classes)
cross_val_score(knn_run_svae, z_run_svae.detach().numpy(), run_classes)

array([0.62980769, 0.62980769, 0.63942308, 0.64251208, 0.63768116])

In [108]:
# KNN classifier on trials using vae 
knn_run_vae = KNeighborsClassifier(n_neighbors=3)
knn_run_vae.fit(z_run.detach().numpy(), run_classes)
cross_val_score(knn_run_vae, z_run.detach().numpy(), run_classes)

array([0.62980769, 0.65865385, 0.63461538, 0.647343  , 0.63285024])

In [10]:
# Swimming dataset
swim_dir = "../runs/swimming/vae_1"
swim_dir_svae = "../runs/swimming/vae_3"

if __name__ == "__main__":
    swim_data = MotionCaptureDataset("126", test=True)

    vae = VariationalAutoencoder(
        latent_dim=2, **model_args
    )
    svae = SphericalVAE(
        latent_dim=3, **model_args
    )

    vae_state_dict = torch.load(f"{swim_dir}/best_vae.pt", map_location=torch.device('cpu')  )
    vae.load_state_dict( vae_state_dict )

    svae_state_dict = torch.load(f"{swim_dir_svae}/best_svae.pt", map_location=torch.device('cpu')  )
    svae.load_state_dict( svae_state_dict )


In [13]:
import re
pattern = re.compile("_(\d+):")

swim_dict = {1:"Back Stroke", 2:"Back Stroke", 3:"Breast Stroke", 4:"Breast Stroke", 5:"Breast Stroke", 6:"Fly Stroke", 7:"Fly Stroke", 8:"Fly Stroke", 9:"Fly Stroke", 10:"Free Style", 11:"Free Style", 12:"Free Style", 13:"Motorcycle", 14:"Range of Motion"}
swim_classes = [int(pattern.search(s).group(1)) for s in swim_data.labels]
swim_classes_labels = [swim_dict[i] for i in swim_classes]

z_swim = vae(swim_data.X)['z']
y_swim = swim_classes_labels

In [14]:
# KNN classifier on swim trials using vae 
knn_swim_vae = KNeighborsClassifier(n_neighbors=3)
knn_swim_vae.fit(z_swim.detach().numpy(), y_swim)
cross_val_score(knn_swim_vae, z_swim.detach().numpy(), y_swim)

array([0.99676026, 0.99568035, 0.99676026, 0.99513776, 0.99837925])

In [15]:
knn_swim_vae = KNeighborsClassifier(n_neighbors=3)
knn_swim_vae.fit(svae(swim_data.X)['z'].detach().numpy(), y_swim)
cross_val_score(knn_swim_vae, svae(swim_data.X)['z'].detach().numpy(), y_swim)

array([0.99946004, 0.99838013, 0.99892009, 0.9989195 , 0.9989195 ])